In [1]:
import visualizer
import main_ver01 as main
import pandas as pd

In [4]:
comp_urls = main.getLeagueLinks(main_url='https://www.whoscored.com/')

In [3]:
comp_urls

['https://www.whoscored.com/Regions/252/Tournaments/2/England-Premier-League',
 'https://www.whoscored.com/Regions/108/Tournaments/5/Italy-Serie-A',
 'https://www.whoscored.com/Regions/206/Tournaments/4/Spain-LaLiga',
 'https://www.whoscored.com/Regions/81/Tournaments/3/Germany-Bundesliga',
 'https://www.whoscored.com/Regions/74/Tournaments/22/France-Ligue-1',
 'https://www.whoscored.com/Regions/177/Tournaments/21/Portugal-Liga-NOS',
 'https://www.whoscored.com/Regions/155/Tournaments/13/Netherlands-Eredivisie',
 'https://www.whoscored.com/Regions/182/Tournaments/77/Russia-Premier-League',
 'https://www.whoscored.com/Regions/31/Tournaments/95/Brazil-Brasileir%C3%A3o',
 'https://www.whoscored.com/Regions/233/Tournaments/85/USA-Major-League-Soccer',
 'https://www.whoscored.com/Regions/225/Tournaments/17/Turkey-Super-Lig',
 'https://www.whoscored.com/Regions/252/Tournaments/7/England-Championship',
 'https://www.whoscored.com/Regions/253/Tournaments/20/Scotland-Premiership',
 'https://www

In [5]:
# teams, match_links = main.getMatchLinks(main_url='https://www.whoscored.com/', 
# #                                         comp_url=comp_urls[0],
#                                         comp_url = 'https://www.whoscored.com/Regions/252/Tournaments/2/England-Premier-League',
#                                        )

In [12]:
team_links = main.getTeamLinks(f'{target_team}', match_links)
team_links

In [14]:
import re
compiler = re.compile(r'\d+')
sorted(team_links, key=lambda s: compiler.search(s).group())

['https://www.whoscored.com/Matches/1283367/Live/Netherlands-Eredivisie-2018-2019-Vitesse-Ajax',
 'https://www.whoscored.com/Matches/1283484/Live/Netherlands-Eredivisie-2018-2019-FC-Utrecht-Ajax',
 'https://www.whoscored.com/Matches/1283496/Live/Netherlands-Eredivisie-2018-2019-Ajax-SC-Heerenveen',
 'https://www.whoscored.com/Matches/1283509/Live/Netherlands-Eredivisie-2018-2019-Ajax-VVV-Venlo',
 'https://www.whoscored.com/Matches/1283598/Live/Netherlands-Eredivisie-2018-2019-FC-Emmen-Ajax',
 'https://www.whoscored.com/Matches/1283682/Live/Netherlands-Eredivisie-2018-2019-FC-Groningen-Ajax',
 'https://www.whoscored.com/Matches/1343400/Live/Europe-Champions-League-2018-2019-Ajax-Real-Madrid',
 'https://www.whoscored.com/Matches/1343408/Live/Europe-Champions-League-2018-2019-Real-Madrid-Ajax',
 'https://www.whoscored.com/Matches/1368025/Live/Europe-Champions-League-2018-2019-Ajax-Juventus',
 'https://www.whoscored.com/Matches/1368026/Live/Europe-Champions-League-2018-2019-Juventus-Ajax',

In [2]:
# match_data = main.getTeamData(team_links)
match_url = "https://www.whoscored.com/Matches/1592152/Live/Europe-Champions-League-2021-2022-Benfica-Barcelona"
match_data = main.getMatchData(url=match_url)

Region: Europe, League: Champions League, Season: 2021/2022, Match Id: 1592152


In [2]:
teamName = 'barcelona'
season = input(str("please input season."))
period = input(str('Please input this game Period'))

url = input(str("match URL ..."))
match_data = main.getMatchData(url=url)

import pickle
# Write
# with open(file=f"/work/assets/whoscored/{teamName}/match/{season}/matchData/#{period}.json",mode="wb") as file:
#     pickle.dump(match_data,file)

# Read
with open(file=f"/work/assets/whoscored/{teamName}/match/{season}/matchData/#{period}.json", mode="rb") as file:
    match_data = pickle.load(file)


matchId = match_data['matchId']
homeId = match_data['home']['teamId']
homeFormation = match_data['home']['formations'][0]['formationName']

awayId = match_data['away']['teamId']
awayFormation = match_data['away']['formations'][0]['formationName']

matches_df = main.createMatchesDF(match_data)

homeName = matches_df['home'][matchId]['name']
homeScore = matches_df['home'][matchId]['scores']['fulltime']
homeAge = matches_df['home'][matchId]['averageAge']

awayName = matches_df['away'][matchId]['name']
awayScore = matches_df['away'][matchId]['scores']['fulltime']
awayAge = matches_df['away'][matchId]['averageAge']


events_df = main.createEventsDF(match_data)
events_df = main.addEpvToDataFrame(events_df)

import os
path = f"/work/assets/whoscored/{teamName}/match/{season}/eventsData/"
if not os.path.exists(path):
    os.makedirs(path)

events_df.to_csv(path + f'{season}#{period}.csv')

team_players_dict = {}
player_ratings = {}
for venue in ['home','away']:
    team_players_dict[venue] = {}
    for player in matches_df[venue][matchId]['players']:
        team_players_dict[venue][(player['playerId'],player["name"])] = player['position']
        try:
            player_ratings[player['name']] = player['stats']['ratings']
        except KeyError:
            pass

import pickle

# with open(f"/work/assets/whoscored/{teamName}/ids/{season}/{season}#{period}.json", "wb") as jsonFile:
#     pickle.dump(team_players_dict,jsonFile)
            
            
df_total = pd.io.json.json_normalize(match_data)
venue_list = df_total['venueName']

df_opponent = df_total[['home.name', 'away.name']]
opponent_list = []
for idx, row in df_opponent.iterrows():
    if row['home.name']==f'{teamName.title()}':
        opponent_list.append(row['away.name'])
    else:
        opponent_list.append(row['home.name'])
import os
for i, data in enumerate(df_total['events']):
    df_events = pd.io.json.json_normalize(data)
    df_events['venue'] = venue_list[i]
    df_events['opponent'] = opponent_list[i]
    path = f"/work/assets/whoscored/{teamName}/match/{season}/eventsData/"
    if not os.path.exists(path):
        os.makedirs(path)
        
    df_events.to_csv(path + f'{season}#{period}.csv')
    

please input season. 2223
Please input this game Period 1
match URL ... https://www.whoscored.com/Matches/1650630/Live/Spain-LaLiga-2022-2023-Barcelona-Rayo-Vallecano


Region: Spain, League: LaLiga, Season: 2022/2023, Match Id: 1650630


/work/scrayper/whoscored/main_ver01.py:376: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matches_df = matches_df.append(matches_dict, ignore_index=True)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
<ipython-input-2-6bc3be928538>:63: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df_total = pd.io.json.json_normalize(match_data)
<ipython-input-2-6bc3be928538>:75: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df_events = pd.io.json.json_normalize(data)


In [21]:
events_df["qualifiers"][736]

[{'type': {'value': 103, 'displayName': 'GoalMouthZ'}, 'value': '25'},
 {'type': {'value': 56, 'displayName': 'Zone'}, 'value': 'Center'},
 {'type': {'value': 147, 'displayName': 'BlockedY'}, 'value': '42.6'},
 {'type': {'value': 146, 'displayName': 'BlockedX'}, 'value': '89.8'},
 {'type': {'value': 73, 'displayName': 'MissLeft'}},
 {'type': {'value': 29, 'displayName': 'Assisted'}},
 {'type': {'value': 55, 'displayName': 'RelatedEventId'}, 'value': '495'},
 {'type': {'value': 15, 'displayName': 'Head'}},
 {'type': {'value': 233, 'displayName': 'OppositeRelatedEvent'},
  'value': '303'},
 {'type': {'value': 24, 'displayName': 'SetPiece'}},
 {'type': {'value': 102, 'displayName': 'GoalMouthY'}, 'value': '73.1'},
 {'type': {'value': 154, 'displayName': 'IntentionalAssist'}},
 {'type': {'value': 17, 'displayName': 'BoxCentre'}}]

In [5]:
import pickle
## Write
season = "2122"
target_team = "barcelona"
gw = input("please input game week")
with open(file=f"/work/assets/whoscored/{target_team}/match/{season}/matchData/#{gw}.json",mode="wb") as file:
    pickle.dump(match_data,file)

please input game week 6


In [7]:
# import pandas as pd
import pandas as pd
df_total = pd.io.json.json_normalize(match_data)
len(match_data)

<ipython-input-7-b311ff7a7fdd>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df_total = pd.io.json.json_normalize(match_data)


36

In [8]:
venue_list = df_total['venueName']

df_opponent = df_total[['home.name', 'away.name']]
opponent_list = []
for idx, row in df_opponent.iterrows():
    if row['home.name']==f'{target_team.title()}':
        opponent_list.append(row['away.name'])
    else:
        opponent_list.append(row['home.name'])
opponent_list

['Levante']

In [9]:
import os
for i, data in enumerate(df_total['events']):
    df_events = pd.io.json.json_normalize(data)
    df_events['venue'] = venue_list[i]
    df_events['opponent'] = opponent_list[i]
    path = f'/work/assets/whoscored/{target_team}/match/{season}/'
    if not os.path.exists(path):
        os.makedirs(path)
#     df_events.to_csv(path + f'2122#{opponent_list[i]}.csv')
#     df_events.to_csv(path + f'2021#{gw}.csv')
    if season == "2122":
        df_events.to_csv(f"/work/assets/whoscored/{target_team}/match/{season}/eventsData/{season}#{gw}.csv")
    else:
        df_events.to_csv(f"/work/assets/whoscored/{target_team}/match/{season}/{season}#{gw}.csv")
#     df_events.to_csv(path + f'2021#{i}.csv')
    
    

<ipython-input-9-681db81888ad>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df_events = pd.io.json.json_normalize(data)


In [11]:
matchId = match_data['matchId']
homeId = match_data['home']['teamId']
homeFormation = match_data['home']['formations'][0]['formationName']

awayId = match_data['away']['teamId']
awayFormation = match_data['away']['formations'][0]['formationName']

matches_df = main.createMatchesDF(match_data)

homeName = matches_df['home'][matchId]['name']
homeScore = matches_df['home'][matchId]['scores']['fulltime']
homeAge = matches_df['home'][matchId]['averageAge']

awayName = matches_df['away'][matchId]['name']
awayScore = matches_df['away'][matchId]['scores']['fulltime']
awayAge = matches_df['away'][matchId]['averageAge']


# events_df = main.createEventsDF(match_data)

team_players_dict = {}
player_ratings = {}
for venue in ['home','away']:
    team_players_dict[venue] = {}
    for player in matches_df[venue][matchId]['players']:
        team_players_dict[venue][(player['playerId'],player["name"])] = player['position']
        try:
            player_ratings[player['name']] = player['stats']['ratings']
        except KeyError:
            pass

import pickle

with open(f"/work/assets/whoscored/{teamName}/ids/{season}/{season}#{gw}.json", "wb") as jsonFile:
    pickle.dump(team_players_dict,jsonFile)
    

In [ ]:
matchIds = matches_df.index.values

In [ ]:
ids = pd.read_csv('/work/assets/whoscored/barcelona/ids/player_id.csv')

In [ ]:
playerIds = events_df['playerId']

In [ ]:
e_df = events_df[(events_df['isGoal']==True)|(events_df['playerId']==11119.0)]
types = []
for row in e_df['type']:
    rn = row['displayName']
    types.append(rn)
e_df['detail'] = types
e_df = e_df[['playerId', 'type', 'second', 'detail']]
e_df = e_df[(e_df['detail']=='Pass')|(e_df['detail']=='Goal')]
ids = e_df['playerId']

In [ ]:
next_df = e_df.shift(-1)
next_df = next_df.rename(columns={'type':'type_next', 'second':'minute_next', 'detail':'detail_next'})

df = pd.merge(e_df,next_df)

In [6]:
events_df["EPV"].isna()

0       True
1       True
2       True
3       True
4       True
        ... 
1514    True
1515    True
1516    True
1517    True
1518    True
Name: EPV, Length: 1519, dtype: bool